In [19]:
%run 'init.ipynb'


#example
material = "SS_OuterCryostat"
isotope = "Co60"

rootfile = '/userdata/arocchetti/XENONnT_2019-11-04/'+ material + '/' + isotope +'/'+ 'output_'+ material+'_' + isotope +'_1_Sort.root'

data = get_xyz(rootfile)

In [2]:
def get_xyz_root(rootfile):
    file = uproot.open(rootfile)
    tree = file["events/events"]
    xp = tree["xp_pri"].array()
    df = tree.pandas.df([ "xp_pri", "yp_pri", "zp_pri"])
    df.columns = ['X', 'Y', 'Z_uc'] #rename 
    #Add some variables
    df['R'] = np.sqrt(df.X*df.X+ df.Y*df.Y)
    df['R2'] = df.R*df.R
    offset = 1488/2
    df['Z'] = df.Z_uc+ offset 
    return df

In [3]:
def print_volume(data):
    plt.hist2d(data.r, data.Z, bins = (300,300), range = ((0, 1000), (-1500, 1700)), 
    norm = LogNorm())
    plt.ylabel('z [mm]')
    plt.xlabel('r [mm]')
    plt.title(material)
    plt.tight_layout()
    plt.title(material)
    plt.show()

def print_xy(data):
    plt.hist2d(data.X, data.Y, bins = (300,300), range = ((-950, 950), (-950, 950)), 
    norm = LogNorm())
    plt.ylabel('x [mm]')
    plt.xlabel('y [mm]')
    plt.title(material)
    plt.tight_layout()
    plt.title(material)
    plt.show()
    
def scatter(data):
    import matplotlib.lines as mlines
    plt.scatter(data.R, data.Z, marker = '.', s=2,label=material )
    c=1
    plt.xlim(-100,1200)
    plt.ylim(-1500, 1700)
    plt.ylabel('z [mm]')
    plt.xlabel('r [mm]')
    plt.tight_layout()
    plt.legend(markerscale=10, bbox_to_anchor=(1.05, 1), 
               loc='upper left', fontsize='xx-large')

In [21]:
a = 24.82 + 25.14 + 72.53 + 7.19 + 95.85 
print(a)

225.53


In [31]:
def get_parameters(material):
    if material == "OuterCryostatShell":
        param = {
        'ids' : '1ton-values',
        'mass' : 569400, 
        'U238Pb206' : 2.4,
        'err_U238Pb206' : 0.7,
        'Rn226' : 0.64, 
        'err_Rn226' : 0,
        'Co60' : 9.7,
        'err_Co60' : 0.8,
        'K40' : 2.7, 
        'err_K40' : 0,
        'Cs137' : 0.64, 
        'err_Cs137' : 0, 
        'Th228Pb208' : 0.36, 
        'err_Th228Pb208' : 0, 
        'U235Pb207' : 0.11,
        'err_U235Pb207' : 0.03,
        }
           
    if material == "OuterCryostat_Elongation":
        param = {
        'ids' : "SS_NiT_C5",
        'mass' : 61500 ,
        'U238Pb206': 4,
        'err_U238Pb206' : 2,
        'Rn226' : 1.34 ,
        'err_Rn226' : 0.09,
        'Co60' : 0.61,
        'err_Co60' :0.05, 
        'K40' :1.4 ,
        'err_K40' :0.2,
        'Cs137' : 0.034,
        'err_Cs137' : 0.017,
        'Th228Pb208' : 0.57,
        'err_Th228Pb208' : 0.06,
        'U235Pb207' : 0.31,
        'err_U235Pb207' : 0, 
        }
        
    if material == "OuterCryostat_flanges":
        param = {
        'mass':413220,
        'ids':"XENON1T",
        'U238Pb206':1.4,
        'err_U238Pb206':0.4,
        'Rn226':0.4 ,
        'err_Rn226': 0,
        'Co60':37.3,
        'err_Co60': 0.9,
        'K40':5.6,
        'err_K40': 0,
        'Cs137':1.5,
        'err_Cs137':0,
        'Th228Pb208': 4.5,
        'err_Th228Pb208':0.6,
        'U235Pb207':0.06,
        'err_U235Pb207': 0.02,
        }
    if material == "SS_InnerCryostatShell":
        param = {
         'mass' : 45247 ,#gr
         'ids' : "SS_NiT_C1",
         'U238Pb206' : 3.7,
         'err_U238Pb206' : 0.6,
         'Rn226' : 0.3 ,
         'err_Rn226' : 0.1,
         'Co60' : 2.36,
         'err_Co60' : 0.21,
         'K40' : 1.6,
         'err_K40': 0.6,
         'Cs137' : 0.21,
         'err_Cs137' : 0,
         'Th228Pb208' : 0.5,
         'err_Th228Pb208' : 0.1,
         'U235Pb207' : 0.7,
         'err_U235Pb207' : 0.3 ,
        }
    
    if material == "SS_InnerCryostatFlanges":
        param = { 
        'mass' : 227500, #gr
        'ids' : "XENON1T",
        'U238Pb206' : 1.4,
        'err_U238Pb206' : 0.4,
        'Rn226' : 0.4 ,
        'err_Rn226' : 0,
        'Co60' : 37.3,
        'err_Co60' : 0.9,
        'K40' : 5.6,
        'err_K40' : 0,
        'Cs137' : 1.5,
        'err_Cs137' : 0,
        'Th228Pb208' : 4.5,
        'err_Th228Pb208' : 0.6,
        'U235Pb207' :  0.06,
        'err_U235Pb207' : 0.02,
       }
        
    if material == "SS_InnerCryostat_bottomDome":
        param = {
        'ids' : "SS_NiT_C5",
        'mass' :86800, #g
        'U238Pb206': 4,
        'err_U238Pb206' : 2,
        'Rn226' : 1.34 ,
        'err_Rn226' : 0.09,
        'Co60' : 0.61,
        'err_Co60' :0.05, 
        'K40' :1.4 ,
        'err_K40' :0.2,
        'Cs137' : 0.034,
        'err_Cs137' : 0.017,
        'Th228Pb208' : 0.57,
        'err_Th228Pb208' : 0.06,
        'U235Pb207' : 0.31,
        'err_U235Pb207' : 0, #up
        }
            
    
    if material =="OuterCryostatReflector":
        param = {
        'ids' : "ePTFE-A",
        'mass' : 18660, #gr from mc
        'U238Pb206' : 0.27,
        'err_U238Pb206' : 0.12,
        'Co60' :  0,
        'err_Co60' : 0,
        'K40' : 0,
        'err_K40' : 0,
        'Cs137' :  0,
        'err_Cs137' :  0,
        'Th228Pb208' : 0,
        'err_Th228Pb208' : 0,
        'U235Pb207' :  0,
        'err_U235Pb207' : 0,
        'Th235' : 0.12,
        'err_Th235' : 0.4,
        }
        
    if material == "SS_BellPlate":
        param = {
         'mass' : 63668.81, #gr
         'ids' : "SS_NiT_C1",
         'U238Pb206' : 3.7,
         'err_U238Pb206' : 0.6,
         'Rn226' : 0.3 ,
         'err_Rn226' : 0.1,
         'Co60' : 2.36,
         'err_Co60' : 0.21,
         'K40' : 1.6,
         'err_K40': 0.6,
         'Cs137' : 0.21,
         'err_Cs137' : 0,
         'Th228Pb208' : 0.5,
         'err_Th228Pb208' : 0.1,
         'U235Pb207' : 0.7,
         'err_U235Pb207' : 0.3 ,
        }   
        
    if material == "SS_BellSideWall":
        param = {
        'ids' : "SS_NiT_C5",
        'mass' :33354.05, #gr
        'U238Pb206': 4,
        'err_U238Pb206' : 2,
        'Rn226' : 1.34 ,
        'err_Rn226' : 0.09,
        'Co60' : 0.61  ,
        'err_Co60' :0.05, 
        'K40' :1.4 ,
        'err_K40' :0.2,
        'Cs137' : 0.034,
        'err_Cs137' : 0.017,
        'Th228Pb208' : 0.57,
        'err_Th228Pb208' : 0.06,
        'U235Pb207' : 0.31,
        'err_U235Pb207' : 0, #up
        }
        
    if material == "SS_BellSideWallBottomLip":
        param = { 
         'mass' : 7127.38, #gr
         'ids' :"SS_Nit_E1",
         'U238Pb206' :2.5,
         'err_U238Pb206' : 0.3,
         'Rn226' : 0.6 ,
         'err_Rn226' : 0.1,
         'Co60' :0.4,
         'err_Co60' :0.1,
         'K40' :0.24,
         'err_K40' : 0,
         'Cs137' : 0.16,
         'err_Cs137' : 0,
         'Th228Pb208' : 0.4,
         'err_Th228Pb208' : 0.1,
         'U235Pb207' : 0.56,
         'err_U235Pb207' : 0,
        }
        
    if material == "PmtTpc":
        param = {
        'ids' : "average 1T/nT----missing U",
        'mass' :494, 
        'U238Pb206': 5,
        'err_U238Pb206' : 1.6,
        'Rn226' : 0.24 ,
        'err_Rn226' : 0.08,
        'Co60' : 0.47  ,
        'err_Co60' :0.13, 
        'K40' : 6.6 ,
        'err_K40' :1.4,
        'Cs137' : 0.08,
        'err_Cs137' : 0.05,
        'Th228Pb208' : 0.23,
        'err_Th228Pb208' : 0.09,
        'U235Pb207' : 0,
        'err_U235Pb207' : 0, #up
        }
    
    if material =="Copper_TopRing":
        param = {
        'ids' : "average 1T/nT----missing U",
        'mass' : 53395.39, #gr 
        'U238Pb206': 0.33,
        'err_U238Pb206' : 0,
        'Rn226' : 0.17 ,
        'err_Rn226' : 0,
        'Co60' : 0.031  ,
        'err_Co60' :0, 
        'K40' : 0.45,
        'err_K40' :0.14,
        'Cs137' : 0.054,
        'err_Cs137' : 0,
        'Th228Pb208' : 0.18,
        'err_Th228Pb208' : 0.05,
        'U235Pb207' : 0,
        'err_U235Pb207' : 0, #up
        }
    
    if material == "Copper_LowerRing":
        param = {
        'ids' : "average 1T/nT----missing U",
        'mass' : 84971.21, #gr
        'U238Pb206': 0.33,
        'err_U238Pb206' : 0,
        'Rn226' : 0.17 ,
        'err_Rn226' : 0,
        'Co60' : 0.031  ,
        'err_Co60' :0, 
        'K40' : 0.45,
        'err_K40' :0.14,
        'Cs137' : 0.054,
        'err_Cs137' : 0,
        'Th228Pb208' : 0.18,
        'err_Th228Pb208' : 0.05,
        'U235Pb207' : 0,
        'err_U235Pb207' : 0, #up
        }
       
    if material == "Teflon_Pillar_":
        param = {
         'mass' : 28956.96, #gr
         'ids' : "PTFE_Pillars",
         'U238Pb206' : 0.14,
         'err_U238Pb206' : 0.07,
         'Rn226' : 0.043,
         'err_Rn226' : 0.0011,
         'Co60' : 0,
         'err_Co60' : 0,
         'K40' : 0.42,
         'err_K40' : 0,
         'Cs137' : 0.013,
         'err_Cs137' : 0,
         'Th228Pb208' : 0.042,
         'err_Th228Pb208' : 0,
         'U235Pb207' : 0.049,
         'err_U235Pb207' : 0,
        }
        
        
    if material == "SS_Electrodes":
        param = {
        'mass' : 1760+1914+1595+1508+1207, #gr
        'ids' :"SS_Nit_E1",
         'U238Pb206' :2.5,
         'err_U238Pb206' : 0.3,
         'Rn226' : 0.6 ,
         'err_Rn226' : 0.1,
         'Co60' :0.4,
         'err_Co60' :0.1,
         'K40' :0.24,
         'err_K40' : 0,
         'Cs137' : 0.16,
         'err_Cs137' : 0,
         'Th228Pb208' : 0.4,
         'err_Th228Pb208' : 0.1,
         'U235Pb207' : 0.56,
         'err_U235Pb207' : 0,
        } 
        
    if material == "Teflon_BottomTPC":
        param = {
         'mass': 225.53,
         'ids': "worse between PTFE_Holders and PTFE_Pillars",
         'U238Pb206': 0.14,
         'err_U238Pb206':  0.07,
         'Rn226': 0.064,
         'err_Rn226': 0,
         'Co60': 0,
         'err_Co60': 0,
         'K40':  7,
         'err_K40':  1,
         'Cs137':  0.22,
         'err_Cs137': 0,
         'Th228Pb208': 0.042,
         'err_Th228Pb208':  0,
         'U235Pb207':  0.070,
         'err_U235Pb207': 0,
        }
        
    if material == "Teflon_TPC_Sliding":
        param = { 
        'mass' : 7571.18, #kg
         'ids' : "PTFE_SlidingReflector",
         'U238Pb206' : 0.15,
         'err_U238Pb206' : 0.07,
         'Rn226' : 0.150 ,
         'err_Rn226' : 0.030,
         'Co60' :  0,
         'err_Co60' : 0,
         'K40' : 0.080,
         'err_K40' : 0.030,
         'Cs137' : 0.046,
         'err_Cs137' :  0,
         'Th228Pb208' : 0.079,
         'err_Th228Pb208' : 0,
         'U235Pb207' :  0.065,
         'err_U235Pb207' : 0,
        }
        
    if material == "Teflon_TPC_Fixed":
        param = { 
        'mass' : 11623.4, #gr
         'ids' : "PTFE_Ref2",
         'U238Pb206' : 0.032,
         'err_U238Pb206' :  0,
         'Rn226' :  0.032 ,
         'err_Rn226' : 0,
         'Co60' : 0,
         'err_Co60' :  0,
         'K40' : 8,
         'err_K40' :  1,
         'Cs137' :  0.066,
         'err_Cs137' :  0,
         'Th228Pb208' : 0.089,
         'err_Th228Pb208' : 0,
         'U235Pb207' :  0.078,
         'err_U235Pb207' :  0,
        }
        
    if material == "Copper_FieldGuard_": 
        param = {
        'ids' : "CU_FS01",
         'mass' : 170356.48, #gr
         'U238Pb206' : 0.03,
         'err_U238Pb206' : 0.01,
         'Rn226' :  0 ,
         'err_Rn226' : 0,
         'Co60' : 0,
         'err_Co60' :  0,
         'K40' :  0,
         'err_K40' : 0,
         'Cs137' : 0,
         'err_Cs137' : 0,
         'Th228Pb208' : 0.010,
         'err_Th228Pb208' : 0.004,
         'U235Pb207' :  0,
         'err_U235Pb207' : 0,
        }
        
    if material == "Copper_FieldShaperRing_":
        param = {
        'ids' : "CU_FSW",
        'mass' :8498.16,#gr
        'U238Pb206' : 0,
        'err_U238Pb206' : 0,
        'Rn226' : 0 ,
        'err_Rn226' : 0,
        'Co60': 0,
        'err_Co60' : 0,
        'K40' : 0,
        'err_K40' : 0,
        'Cs137' : 0,
        'err_Cs137' : 0,
        'Th228Pb208' : 0,
        'err_Th228Pb208' :0,
        'U235Pb207' : 0,
        'err_U235Pb207' : 0  ,  
        }
        
    if material == "Top_PTFE_Frame":
        param = {
        'ids' : "unknown",
        'mass' : 8498.16,#gr
        'U238Pb206' : 0,
        'err_U238Pb206' : 0,
        'Rn226' : 0 ,
        'err_Rn226' : 0,
        'Co60': 0,
        'err_Co60' : 0,
        'K40' : 0,
        'err_K40' : 0,
        'Cs137' : 0,
        'err_Cs137' : 0,
        'Th228Pb208' : 0,
        'err_Th228Pb208' :0,
        'U235Pb207' : 0,
        'err_U235Pb207' : 0  ,  
        }
        
    if material == "Copper_BottomPmtPlate":
        param = {
        'ids' : "unknown",
        'mass' : 84971.2,#gr
        'U238Pb206' : 0,
        'err_U238Pb206' : 0,
        'Rn226' : 0 ,
        'err_Rn226' : 0,
        'Co60': 0,
        'err_Co60' : 0,
        'K40' : 0,
        'err_K40' : 0,
        'Cs137' : 0,
        'err_Cs137' : 0,
        'Th228Pb208' : 0,
        'err_Th228Pb208' :0,
        'U235Pb207' : 0,
        'err_U235Pb207' : 0  ,  
        }
                
    if material == "Copper_TopPmtPlate":
        param = {
        'ids' : "unknown",
        'mass' : 53395.39, #gr
        'U238Pb206' : 0,
        'err_U238Pb206' : 0,
        'Rn226' : 0 ,
        'err_Rn226' : 0,
        'Co60' : 0,
        'err_Co60' : 0,
        'K40' : 0,
        'err_K40' : 0,
        'Cs137' : 0,
        'err_Cs137' : 0,
        'Th228Pb208' : 0,
        'err_Th228Pb208': 0,
        'U235Pb207' : 0,
        'err_U235Pb207' : 0,  
        }
        
    return param

In [32]:
material = "Copper_BottomPmtPlate"
p = get_parameters(material)

In [125]:
def get_parameters_dic(material):
    if material == "OuterCryostatShell":
        param = {
        'ids' : '1ton-values',
        'mass' : 569400, 
        'U238Pb206' : 2.4,
        'err_U238Pb206' : 0.7,
        'Rn226' : 0.64, 
        'err_Rn226' : 0,
        'Co60' : 9.7,
        'err_Co60' : 0.8,
        'K40' : 2.7, #ul
        'err_K40' : 0,
        'Cs137' : 0.64, #up
        'err_Cs137' : 0, #up
        'Th228Pb208' : 0.36, #up
        'err_Th228Pb208' : 0, #up
        'U235Pb207' : 0.11,
        'err_U235Pb207' : 0.03,
        }
        return param




In [117]:
def calculate_rate(mass, contamination, err_contamination, m_fv, E):
    time = n_gen / (mass * contamination)
    rate = n_events/(time * m_fv * E)
    alpha = mass / (m_fv * E * n_gen)
    error_rate = alpha * sqrt((n_events * err_contamination)**2 + (contamination*sqrt(n_events))**2)
    
    

In [118]:
def fv(data):
    data['fv-line'] = ((abs(data.Z/634.))**3+abs(data.R/401956.)**3) #add variable
    data = data[data['fv-line']<1]
    data.head(2)
    return data 


In [119]:
def single_scatter(data):
    data = data[data.ns==1]
    return data

In [120]:
def select_typepri(data):
    if isotope =="U238":
        data = data[(data.typepri=='Th234')  |
                    (data.typepri=='Pa234')  |
                    (data.typepri=='U234')   |
                    (data.typepri=='Th230')  |
                    (data.typepri=='U238') 
                   ]
    if isotope =="U235":
        print("check THIS")
        data = data[(data.typepri=='U235')  |
                    (data.typepri=='Ra228') |
                    (data.typepri=='Ac228') |
                    (data.typepri=='Th228') |
                    (data.typepri=='Ra224') |
                    (data.typepri=='Rn220') |
                    (data.typepri=='Po216') | 
                    (data.typepri=='Bi212') |
                    (data.typepri=='Tl208') |
                    (data.typepri=='Pb208') 
                   ] 
    if isotope =="Th228":
        data = data[(data.typepri=='Th228') |
                    (data.typepri=='Ra224') |
                    (data.typepri=='Rn220') |
                    (data.typepri=='Po216') |
                    (data.typepri=='Pb212') |
                    (data.typepri=='Bi212') |
                    (data.typepri=='Tl208') 
                   ] 
    if isotope =="Th232":
        data = data[(data.typepri=='Th232')  |
                    (data.typepri=='Rn228')  |
                    (data.typepri=='Ac228')  
                   ]   
    if isotope =="Rn226":
        data = data[(data.typepri=='Rn226') |
                    (data.typepri=='Rn222') |
                    (data.typepri=='Po218') |
                    (data.typepri=='Po214') |
                    (data.typepri=='Bi214') |
                    (data.typepri=='Po214') |
                    (data.typepri=='Po210') |
                    (data.typepri=='Bi210') |
                    (data.typepri=='Tl216') 
                   ]     
        return data

In [121]:
#example
from matplotlib.ticker import ScalarFormatter


def plot_spectrum(data):
    plt.hist(data.Ed, bins = 300, range = (0, 350), histtype = 'step')
    plt.ylabel('counts')
    plt.xlabel('Ed [keV]')
    plt.yscale('log')
    plt.tight_layout()
    plt.show()
      

In [122]:
def energy(data, min, max):
    data = data[(data.Ed<max) & (data.Ed>min)]
    return data


36


43519
